In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sarlren/isic16")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/isic16


In [4]:
!git clone https://github.com/WuJunde/MedSegDiff.git
!mv MedSegDiff/* ./

Cloning into 'MedSegDiff'...
remote: Enumerating objects: 716, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 716 (delta 340), reused 248 (delta 248), pack-reused 319 (from 2)
Receiving objects: 100% (716/716), 3.90 MiB | 12.08 MiB/s, done.
Resolving deltas: 100% (396/396), done.


In [5]:
!mkdir /kaggle

mkdir: cannot create directory ‘/kaggle’: File exists


In [6]:
!mkdir /kaggle/working

In [7]:
!mkdir /kaggle/working/out

In [8]:
!mkdir /kaggle/working/isic

In [9]:
!cp -r /content/guided_diffusion /content/scripts/guided_diffusion

In [ ]:
import subprocess
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Test_Data/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Training_Data/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Training_GroundTruth/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Test_GroundTruth/* /kaggle/working/isic/", shell=True)
subprocess.call("cp "+path+"/ISBI2016_ISIC_Part3B_Test_GroundTruth.csv /kaggle/working/isic/", shell=True)
subprocess.call("cp "+path+"/ISBI2016_ISIC_Part3B_Training_GroundTruth.csv /kaggle/working/isic/", shell=True)

In [ ]:
!pip install blobfile
!pip install nibabel
!pip install visdom
!pip install torchsummary
!pip install batchgenerators
!pip install mpi4py

In [ ]:
import pdb

In [ ]:
import sys
import os
sys.path.append("/content/scripts")
sys.path.append("./")
from ssl import OP_NO_TLSv1
import nibabel as nib
import sys
import random
import numpy as np
import time
from PIL import Image
import torch.distributed as dist
import torchvision.utils as vutils
from guided_diffusion import dist_util, logger
from guided_diffusion.resample import create_named_schedule_sampler
from guided_diffusion.bratsloader import BRATSDataset, BRATSDataset3D
from guided_diffusion.isicloader import ISICDataset
from guided_diffusion.custom_dataset_loader import CustomDataset
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    args_to_dict,
    add_dict_to_argparser,
)
import torch as th
from guided_diffusion.train_util import TrainLoop
from guided_diffusion.utils import staple
import torchvision.transforms as transforms
from torchsummary import summary

from dataclasses import dataclass

@dataclass
class Args:
  data_name: str
  data_dir: str
  out_dir: str
  image_size: int
  num_channels: int
  class_cond: bool
  num_res_blocks: int
  num_heads: int
  learn_sigma: bool
  use_scale_shift_norm: bool
  attention_resolutions: str
  diffusion_steps: int
  noise_schedule: str
  rescale_learned_sigmas: bool
  rescale_timesteps: bool
  lr: float
  batch_size: int
  patience: int
  min_delta: float
  microbatch: int
  ema_rate: str
  log_interval: int
  save_interval: int
  resume_checkpoint: str
  schedule_sampler: str
  weight_decay: float
  lr_anneal_steps: int
  use_fp16: bool
  fp16_scale_growth: float
  gpu_dev: str
  multi_gpu: str
  in_ch: int
  num_heads_upsample: int
  num_head_channels: int
  resblock_updown: bool
  dpm_solver: bool
  version: str
  channel_mult: str
  dropout: float
  use_checkpoint: bool
  use_new_attention_order: bool
  timestep_respacing: str
  use_kl: bool
  predict_xstart: bool
  model_path: str
  num_ensemble: int
  use_ddim: bool
  clip_denoised: bool

argsdict = dict(
  data_name = 'ISIC',
  data_dir = "/kaggle/working/isic",
  out_dir = "/kaggle/working/out",
  image_size = 256,
  num_channels = 128,
  class_cond = False,
  num_res_blocks = 2,
  num_heads = 1,
  learn_sigma = True,
  use_scale_shift_norm = False,
  attention_resolutions = "16",
  diffusion_steps = 1000,
  noise_schedule = 'linear',
  rescale_learned_sigmas = False,
  rescale_timesteps = False,
  lr = 1e-4,
  batch_size = 4,
  patience = 2,
  min_delta = 0.001,
  microbatch = -1,  # -1 disables microbatches
  ema_rate = "0.9999",  # comma-separated list of EMA values
  log_interval = 100,
  save_interval = 5000,
  resume_checkpoint = None, #"/results/pretrainedmodel.pt"
  schedule_sampler="uniform",
  weight_decay=0.0,
  lr_anneal_steps=400, # used for early stopping
  use_fp16=False,
  fp16_scale_growth=1e-3,
  gpu_dev = "0",
  multi_gpu = None, #"0,1,2"

  # useful for sampling only:
  model_path = "/kaggle/working/out/savedmodel000400.pt",
  num_ensemble = 5,
  use_ddim = False, #originally False
  clip_denoised = True,
)
argsdict.update(model_and_diffusion_defaults())

args = Args(**argsdict)

seed=10
th.manual_seed(seed)
th.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
def train():
    dist_util.setup_dist(args)
    logger.configure(dir = args.out_dir)

    logger.log("creating data loader...")

    if args.data_name == 'ISIC':
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)

        ds = ISICDataset(args, args.data_dir, transform_train)
        args.in_ch = 4
    elif args.data_name == 'BRATS':
        tran_list = [transforms.Resize((args.image_size,args.image_size)),]
        transform_train = transforms.Compose(tran_list)

        ds = BRATSDataset3D(args.data_dir, transform_train, test_flag=False)
        args.in_ch = 5
    else :
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)
        print("Your current directory : ",args.data_dir)
        ds = CustomDataset(args, args.data_dir, transform_train)
        args.in_ch = 4

    datal= th.utils.data.DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("creating model and diffusion...")

    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    if args.multi_gpu:
        model = th.nn.DataParallel(model,device_ids=[int(id) for id in args.multi_gpu.split(',')])
        model.to(device = th.device('cuda', int(args.gpu_dev)))
    else:
        model.to(dist_util.dev())
    schedule_sampler = create_named_schedule_sampler(args.schedule_sampler, diffusion,  maxt=args.diffusion_steps)


    logger.log("training...")
    TrainLoop(
        model=model,
        diffusion=diffusion,
        classifier=None,
        data=data,
        dataloader=datal,
        batch_size=args.batch_size,
        microbatch=args.microbatch,
        lr=args.lr,
        ema_rate=args.ema_rate,
        log_interval=args.log_interval,
        save_interval=args.save_interval,
        resume_checkpoint=args.resume_checkpoint,
        use_fp16=args.use_fp16,
        fp16_scale_growth=args.fp16_scale_growth,
        schedule_sampler=schedule_sampler,
        weight_decay=args.weight_decay,
        lr_anneal_steps=args.lr_anneal_steps,
    ).run_loop()

def visualize(img):
    _min = img.min()
    _max = img.max()
    normalized_img = (img - _min)/ (_max - _min)
    return normalized_img


def sample():
    dist_util.setup_dist(args)
    logger.configure(dir = args.out_dir)

    if args.data_name == 'ISIC':
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_test = transforms.Compose(tran_list)

        ds = ISICDataset(args, args.data_dir, transform_test, mode = 'Test')
        args.in_ch = 4
    elif args.data_name == 'BRATS':
        tran_list = [transforms.Resize((args.image_size,args.image_size)),]
        transform_test = transforms.Compose(tran_list)

        ds = BRATSDataset3D(args.data_dir,transform_test)
        args.in_ch = 5
    else:
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor()]
        transform_test = transforms.Compose(tran_list)

        ds = CustomDataset(args, args.data_dir, transform_test, mode = 'Test')
        args.in_ch = 4

    datal = th.utils.data.DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("creating model and diffusion...")

    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    all_images = []


    state_dict = dist_util.load_state_dict(args.model_path, map_location="cpu")
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        # name = k[7:] # remove `module.`
        if 'module.' in k:
            new_state_dict[k[7:]] = v
            # load params
        else:
            new_state_dict = state_dict

    model.load_state_dict(new_state_dict)

    model.to(dist_util.dev())
    if args.use_fp16:
        model.convert_to_fp16()
    model.eval()
    for _ in range(len(data)):
        b, m, path = next(data)  #should return an image from the dataloader "data"
        c = th.randn_like(b[:, :1, ...])
        img = th.cat((b, c), dim=1)     #add a noise channel$
        if args.data_name == 'ISIC':
            slice_ID=path[0].split("_")[-1].split('.')[0]
        elif args.data_name == 'BRATS':
            # slice_ID=path[0].split("_")[2] + "_" + path[0].split("_")[4]
            slice_ID=path[0].split("_")[-3] + "_" + path[0].split("slice")[-1].split('.nii')[0]

        logger.log("sampling...")

        start = th.cuda.Event(enable_timing=True)
        end = th.cuda.Event(enable_timing=True)
        enslist = []

        for i in range(args.num_ensemble):  #this is for the generation of an ensemble of 5 masks.
            model_kwargs = {}
            start.record()
            sample_fn = (
                diffusion.p_sample_loop_known if not args.use_ddim else diffusion.ddim_sample_loop_known
            )
            # pdb.set_trace() #imbedded debug !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # truncating model output after its internally calculated in dim 2 should fix the issue:
            # https://github.com/SuperMedIntel/MedSegDiff/issues/201
            # fixable by just editing gaussian_diffusion.py:346 to be permissive
            sample, x_noisy, org, cal, cal_out = sample_fn( # broken
                model,
                (args.batch_size, 3, args.image_size, args.image_size), img,
                step = args.diffusion_steps,
                clip_denoised=args.clip_denoised,
                model_kwargs=model_kwargs,
            )

            end.record()
            th.cuda.synchronize()
            print('time for 1 sample', start.elapsed_time(end))  #time measurement for the generation of 1 sample

            co = th.tensor(cal_out)
            if args.version == 'new':
                enslist.append(sample[:,-1,:,:])
            else:
                enslist.append(co)

            if args.debug:
                # print('sample size is',sample.size())
                # print('org size is',org.size())
                # print('cal size is',cal.size())
                if args.data_name == 'ISIC':
                    # s = th.tensor(sample)[:,-1,:,:].unsqueeze(1).repeat(1, 3, 1, 1)
                    o = th.tensor(org)[:,:-1,:,:]
                    c = th.tensor(cal).repeat(1, 3, 1, 1)
                    # co = co.repeat(1, 3, 1, 1)

                    s = sample[:,-1,:,:]
                    b,h,w = s.size()
                    ss = s.clone()
                    ss = ss.view(s.size(0), -1)
                    ss -= ss.min(1, keepdim=True)[0]
                    ss /= ss.max(1, keepdim=True)[0]
                    ss = ss.view(b, h, w)
                    ss = ss.unsqueeze(1).repeat(1, 3, 1, 1)

                    tup = (ss,o,c)
                elif args.data_name == 'BRATS':
                    s = th.tensor(sample)[:,-1,:,:].unsqueeze(1)
                    m = th.tensor(m.to(device = 'cuda:0'))[:,0,:,:].unsqueeze(1)
                    o1 = th.tensor(org)[:,0,:,:].unsqueeze(1)
                    o2 = th.tensor(org)[:,1,:,:].unsqueeze(1)
                    o3 = th.tensor(org)[:,2,:,:].unsqueeze(1)
                    o4 = th.tensor(org)[:,3,:,:].unsqueeze(1)
                    c = th.tensor(cal)

                    tup = (o1/o1.max(),o2/o2.max(),o3/o3.max(),o4/o4.max(),m,s,c,co)

                compose = th.cat(tup,0)
                vutils.save_image(compose, fp = os.path.join(args.out_dir, str(slice_ID)+'_output'+str(i)+".jpg"), nrow = 1, padding = 10)
        ensres = staple(th.stack(enslist,dim=0)).squeeze(0)
        vutils.save_image(ensres, fp = os.path.join(args.out_dir, str(slice_ID)+'_output_ens'+".jpg"), nrow = 1, padding = 10)

In [ ]:
# bugged section: x=x[:,-1:,...]  #loss is only calculated on the last channel, not on the input brain MR image
# becomes:
# x=x[:,-1:,...]\n        if x.shape != model_output.shape:\n            model_output = model_output[:,0,...]
!sed 's/x=x[:,-1:,...]  #loss is only calculated on the last channel, not on the input brain MR image/x=x[:,-1:,...]\n        if x.shape != model_output.shape:\n            model_output = model_output[:,0,...]/' /content/scripts/guided_diffusion/gaussian_diffusion.py

In [ ]:
train()

In [ ]:
!ls /kaggle/working/out

In [ ]:
%cd /kaggle/working/out
from IPython.display import FileLink
FileLink('/kaggle/working/out/optsavedmodel000000.pt')

In [ ]:
!tar -cf models.tar /kaggle/working/out

In [ ]:
!ls /kaggle/working/out

In [ ]:
!tar czvf model_tar2.tar.gz /kaggle/working/out

In [ ]:
!ls -alh

In [ ]:
!tar -tvf ./model_tar2.tar.gz

In [ ]:
!nvidia_smi

In [ ]:
sample()

In [ ]:
!ls /kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/

In [ ]:
!ls /kaggle/working/out
from matplotlib import pyplot as plt
from PIL import Image
display(Image.open('/kaggle/working/out/0010016_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010016_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010016.jpg'))

display(Image.open('/kaggle/working/out/0010336_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010336_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010336.jpg'))

display(Image.open('/kaggle/working/out/0010437_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010437_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010437.jpg'))

display(Image.open('/kaggle/working/out/0010494_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010494_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010494.jpg'))

display(Image.open('/kaggle/working/out/0010574_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010574_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010574.jpg'))